# Data Science Research Methods Report-2

## **Introduction**<br>
The PAMAP2 Physical Activity Monitoring dataset (available here) contains data from 9 participants who participated in 18 various physical activities (such as walking, cycling, and soccer) while wearing three inertial measurement units (IMUs) and a heart rate monitor. This information is saved in separate text files for each subject. The goal is to build hardware and/or software that can determine the amount and type of physical activity performed by an individual by using insights derived from analysing the given dataset. 

In [45]:
import os
import pdb
import tabula
from IPython.display import display
from matplotlib import rcParams
from scipy.stats import ranksums,ttest_ind
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import random
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import  LogisticRegression
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
import itertools
os.chdir("/home/sahil/Downloads/PAMAP2_Dataset/") # Setting up working directory
import warnings
warnings.filterwarnings("ignore")

## Data Cleaning<br>
For tidying up the data :<br>
- We load the data of various subjects and give relevant column names<br>
  for various features. <br>
- The data for all subjects are then stacked together to form one table.<br>
- We remove the 'Orientation' columns because it was mentioned <br>
  in the data report that it is invalid in this data collection.<br>
- Similarly, the rows with Activity ID "0" are also removed as<br>
  it does not relate to any specific activity.<br>
- The missing values are filled up using the linear interpolation method.<br>
- Added a new feature, 'BMI' or Body Mass Index for the 'subject_detail' table<br>
- Additional feature, 'Activity Type' is added to the data which classifies activities <br>
  into 3 classes, 'Light' activity,'Moderate' activity and 'Intense' activity.<br>
  1. Lying,sitting,ironing and standing are labelled as 'light' activities.<br>
  2. Vacuum cleaning,descending stairs,normal walking,Nordic walking and cycling are<br>
     considered as 'Moderate' activities<br>
  3. Ascending stairs,running and rope jumping are labelled as 'Intense' activities.  <br>
  This classification makes it easier to perform hypothesis testing between pair of attributes.


<br>
Given below are functions to give relevant names to the columns and create a<br>
single table containing data for all subjects<br>


In [ ]:
def gen_activity_names():
    # Using this function all the activity names are mapped to their ids
    act_name = {}
    act_name[0] = 'transient'
    act_name[1] = 'lying'
    act_name[2] = 'sitting'
    act_name[3] = 'standing'
    act_name[4] = 'walking'
    act_name[5] = 'running'
    act_name[6] = 'cycling'
    act_name[7] = 'Nordic_walking'
    act_name[9] = 'watching_TV'
    act_name[10] = 'computer_work'
    act_name[11] = 'car driving'
    act_name[12] = 'ascending_stairs'
    act_name[13] = 'descending_stairs'
    act_name[16] = 'vacuum_cleaning'
    act_name[17] = 'ironing'
    act_name[18] = 'folding_laundry'
    act_name[19] = 'house_cleaning'
    act_name[20] = 'playing_soccer'
    act_name[24] = 'rope_jumping'
    return act_name
def generate_three_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    return [x,y,z]
def generate_four_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    w = name +'_w'
    return [x,y,z,w]
def generate_cols_IMU(name):
    # temp
    temp = name+'_temperature'
    output = [temp]
    # acceleration 16
    acceleration16 = name+'_3D_acceleration_16'
    acceleration16 = generate_three_IMU(acceleration16)
    output.extend(acceleration16)
    # acceleration 6
    acceleration6 = name+'_3D_acceleration_6'
    acceleration6 = generate_three_IMU(acceleration6)
    output.extend(acceleration6)
    # gyroscope
    gyroscope = name+'_3D_gyroscope'
    gyroscope = generate_three_IMU(gyroscope)
    output.extend(gyroscope)
    # magnometer
    magnometer = name+'_3D_magnetometer'
    magnometer = generate_three_IMU(magnometer)
    output.extend(magnometer)
    # oreintation
    oreintation = name+'_4D_orientation'
    oreintation = generate_four_IMU(oreintation)
    output.extend(oreintation)
    return output
def load_IMU():
    output = ['time_stamp','activity_id', 'heart_rate']
    hand = 'hand'
    hand = generate_cols_IMU(hand)
    output.extend(hand)
    chest = 'chest'
    chest = generate_cols_IMU(chest)
    output.extend(chest)
    ankle = 'ankle'
    ankle = generate_cols_IMU(ankle)
    output.extend(ankle)
    return output

In [ ]:
def load_subjects(root1='/home/sahil/Downloads/PAMAP2_Dataset/Protocol/subject',
        root2 ='/home/sahil/Downloads/PAMAP2_Dataset/Optional/subject' ):
    cols = load_IMU()
    output = pd.DataFrame()
    for i in range(101,110):
        path1 = root1 + str(i) + '.dat'
        subject= pd.DataFrame()
         
        subject_prot = pd.read_table(path1, header=None, sep='\s+') # subject data from 
         # protocol activities
        subject = subject.append(subject_prot)
        subject.columns = cols 
        subject = subject.sort_values(by='time_stamp') # Arranging all measurements according to
         # time
        subject['id'] = i
        output = output.append(subject, ignore_index=True)
    return output
data = load_subjects()# Add your own location for the data here to replicate the code
# for eg data = load_subjects('filepath')
data = data.drop(data[data['activity_id']==0].index)# Removing rows with activity id of 0
act = gen_activity_names()
data['activity_name'] = data.activity_id.apply(lambda x:act[x])
data = data.drop([i for i in data.columns if 'orientation' in i], axis=1)  # Dropping Orientation  columns
cols_6g = [i for i in data.columns if '_6_' in i] # 6g acceleration data columns
data =  data.drop(cols_6g,axis=1) # dropping 6g acceleration columns
display(data.head())
# Saving transformed data in pickle format becuse it has the fastest read time compared
# to all other formats
data.to_pickle("activity_data.pkl")  # Saving transformed data for future use

In [ ]:
def train_test_split(data,split_size):
    np.random.seed(5)
    msk = np.random.rand(len(data)) < split_size # This code implies (split_size*100)% of the values will be True
    train = data[msk] # Generating training data
    test = data[~msk] # generating testing data  
    return train,test

In [ ]:
def train_test_split_by_subjects(data): # splitting by subjects
    subjects = [i for i in range(101,109)] # Eliminate subject 109  due to less activities
    train_subjects = [101,103,104,105]
    test_subjects = [i for i in subjects if i not in train_subjects]
    train = data[data.id.isin(train_subjects)] # Generating training data
    test = data[data.id.isin(test_subjects)] # generating testing data  
    return train,test

In [1]:
def split_by_activities(data):
   light = ['lying','sitting','standing','ironing'] 
   moderate = ['vacuum_cleaning','descending_stairs','normal_walking',
           'nordic_walking','cycling']
   intense = ['ascending_stairs','running','rope_jumping']
   def split(activity): #  method for returning activity labels for activities
       if activity in light:
           return 'light'
       elif activity in moderate:
           return 'moderate'
       else:
           return 'intense'
   data['activity_type'] = data.activity_name.apply(lambda x:split(x))
   return data
def split_by_activities_6(data):
   main = ['lying','sitting','standing','walking','running','cycling'] 
   others = [i for i in data.activity_name.unqiue() if i not in main]
   sit_stand = ['sitting','standing']
   if activity in main:
       if activity in sit_stand:
           return 'sitting/standing'
       else:
           return activity
   else:
       return 'others'

In [ ]:
def random_subset(data,subset_frac): # For selecting a random subset of data
    np.random.seed(8)
    msk = np.random.rand(len(data)) < subset_frac # This code implies (split_size*100)% of the values will be True
    subset = data[msk] # Generating subset
    return subset

Loading data and doing the train-test split for EDA and Hypothesis testing.

In [ ]:
data = pd.read_pickle("activity_data.pkl")
data = split_by_activities(data)
train,test = train_test_split_by_subjects(data) # train and test data for EDA and hypothesis testing respectively.
subj_det = tabula.read_pdf("subjectInformation.pdf",pages=1) # loading subject detail table from pdf file.
# Eliminating unnecessary columns and fixing the column alignment of the table.
sd = subj_det[0]
new_cols = list(sd.columns)[1:9]
sd = sd[sd.columns[0:8]]
sd.columns = new_cols 
subj_det=sd

Create clean data for use in modelling

In [ ]:
eliminate = ['activity_id','activity_name','time_stamp','id'] # Columns not meant to be cleaned
features = [i for i in data.columns if i not in eliminate]
clean_data = data
clean_data[features] =clean_data[features].interpolate()
display(clean_data.head())

After linear interpolation, the first four values of heart rate are still missing. So we fill that using back fill method.

In [ ]:
clean_data['heart_rate'] = clean_data['heart_rate'].bfill()
display(clean_data.head())

Finally, save the clean data for future use in model prediction

In [ ]:
clean_data.to_pickle("clean_act_data.pkl")

## Exploratory Data Analysis<br>
After labelling the data appropriately, we have selected 4 subjects for training set and <br>
4 subjects for testing set such that the training and testing set have approximately equal size.<br>
In the training set, we perform Exploratory Data Analysis and come up with potential hypotheses. <br>
We then test those hypotheses on the testing set.<br>
50% of data is used for training in this case(Exploratory data analysis) and the rest for testing.

Calculating BMI of the subjects

In [ ]:
height_in_metres = subj_det['Height (cm)']/100
weight_in_kg = subj_det['Weight (kg)']
subj_det['BMI'] =  weight_in_kg/(height_in_metres)**2

### Data Visualizations

* Bar chart for frequency of activities.

In [ ]:
rcParams['figure.figsize'] = 40,25
ax=sns.countplot(x="activity_name",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)# Rotating Text
plt.show()

* 3D scatter plot of chest acceleration coordinates for lying<br>
<br>
  It is expected that vertical chest acceleration will be more while lying due to the<br>
  movements involved and an attempt is made to check this visually over here.

In [ ]:
plt.clf()
train_running = train[train.activity_name=='lying']
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
x = train_running["chest_3D_acceleration_16_x"]
y = train_running["chest_3D_acceleration_16_y"]
z = train_running["chest_3D_acceleration_16_z"]
ax.scatter(x,y,z)
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')
plt.show()

   As we see, there seems to be more variance along the z axis(vertical direction) than the<br>
   x and y axis.

* 3D scatter plot of chest acceleration coordinates for running<br>
<br>
  Since running involves mostly horizontal movements for the chest, we expect<br>
  most of chest acceleration data to lie on the horizontal x amd y axis.

In [ ]:
plt.clf()
train_running = train[train.activity_name=='running']
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
x = train_running["chest_3D_acceleration_16_x"]
y = train_running["chest_3D_acceleration_16_y"]
z = train_running["chest_3D_acceleration_16_z"]
ax.scatter(x,y,z)
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')
plt.show()

  As we expected, for running, most of the points lie along the x and y axis.

* Time series plot of x axis chest acceleration

In [ ]:
plt.clf()
random.seed(4)
train1 = train[train.id==random.choice(train.id.unique())]
sns.lineplot(x='time_stamp',y='chest_3D_acceleration_16_z',hue='activity_name',data=train1)
plt.show()

* Boxplot of rolling mean of vertical chest acceleration

In [ ]:
train['rolling_mean'] = train['chest_3D_acceleration_16_z'].rolling(256).mean()
ax=sns.boxplot(x="activity_name",y="rolling_mean",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)# Rotating Text
plt.show()

* Boxplot of rolling mean of horizontal ankle acceleration along x axis

In [ ]:
train['rolling_mean'] = train['ankle_3D_acceleration_16_x'].rolling(256).mean()
ax=sns.boxplot(x="activity_name",y="rolling_mean",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)# Rotating Text
plt.show()

* Boxplot of rolling mean of horizontal ankle acceleration along y axis 

In [ ]:
train['rolling_mean'] = train['ankle_3D_acceleration_16_y'].rolling(256).mean()
ax=sns.boxplot(x="activity_name",y="rolling_mean",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)# Rotating Text
plt.show()

* Boxplot of heart rate grouped by activity type. 

In [ ]:
rcParams['figure.figsize'] = 15,10
ax=sns.boxplot(x="activity_type",y="heart_rate",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=0)# Rotating Text
plt.show()

 1. We observe that moderate and intense activities have higher heart rate than<br>
    light activities as expected.<br>
 2. There doesn't seem to be much seperation between moderate and intesne activity<br>
    heart rate.

* Boxplot of heart rate grouped by activity. 

In [ ]:
rcParams['figure.figsize'] = 40,25
ax=sns.boxplot(x="activity_name",y="heart_rate",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)# Rotating Text
plt.show()

  1.  Most of the activities have a skewed distribution for heart rate.<br>
  2. 'Nordic_walking','running' and 'cycling' have a lot of outliers on the lower side.<br>
  3.  Activities like 'lying','sitting' and 'standing' have a lot of outliers on the upper side.

* Boxplot of hand temperature grouped by activity type.

In [ ]:
rcParams['figure.figsize'] = 15,10
ax=sns.boxplot(x="activity_type",y="hand_temperature",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=0)
plt.show()

1. Hand temperature of moderate activitie have a lot of outliers on the lower side.<br>
2. There doesn't seem to be much difference in temperatures between activities.

* Boxplot of hand temperature grouped by activity.

In [ ]:
rcParams['figure.figsize'] = 40,25
ax=sns.boxplot(x="activity_name",y="hand_temperature",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)# Rotating Text
plt.show()

1. Hand temperature data of 'playing_soccer' seems to have a very pronounced positive skew.<br>
2. "car_driving" and "watching_tv" have the least dispersion in hand temperature.

* Boxplot of ankle temperature grouped by activity_type

In [ ]:
rcParams['figure.figsize'] = 15,10
ax=sns.boxplot(x="activity_type",y="ankle_temperature",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=0) 
plt.show()

1. Ankle temperature of light and moderate activitie have  outliers on the lower side.<br>
2. There doesn't seem to be much difference in temperatures between activities.

* Boxplot of ankle temperature grouped by activity

In [ ]:
rcParams['figure.figsize'] = 40,25
ax=sns.boxplot(x="activity_name",y="ankle_temperature",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45) # Rotating Text
plt.show()

1. For ankle temperature, 'playing_soccer' has the least dispersed distribution.<br>
2. Outliers are mostly present in 'vacuum_cleaning' on the lower side. 

* Boxplot of chest temperature grouped by activity_type

In [ ]:
rcParams['figure.figsize'] = 15,10
ax=sns.boxplot(x="activity_type",y="chest_temperature",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=0) 
plt.show()

1. For chest temperatures, only the 'intense' activity type has an outlier.<br>
2. For this feature as well, there doesn't seem to be much difference between <br>
   temperatures.

* Boxplot of chest temperature grouped by activity.

In [ ]:
rcParams['figure.figsize'] = 40,25
ax=sns.boxplot(x="activity_name",y="chest_temperature",data=train)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45) # Rotating Text
plt.show()

1. Most of the activities seem to have a skewed distribution for chest temperature.<br>
2. 'car_driving' and 'watching_tv' seem to have the least dispersed distribution.

* A joint plot trying to investigate possibility of correlation between heart rate <br>
  and chest temperature.

In [ ]:
plt.clf()
rcParams['font.size'] = 20 # Setting the text and number font size
g = sns.JointGrid(data=train, x="heart_rate", y="chest_temperature",
                  height=10,ratio=3)
g.plot_joint(sns.scatterplot,palette='colorblind')
g.plot_marginals(sns.histplot)
# ax.set_xticklabels(ax.get_xticklabels(),rotation=45) # Rotating Text
plt.show()

1. From the scatter plot, we see that there does not seem to be a correlation between<br>
   the two variables.<br>
2. The respective histograms indicate that both the features considered have <br>
   a multi-modal distribution.

### Descriptive Statistics<br>
Subject Details

In [ ]:
display(subj_det)

Mean of heart rate and temperatures for each activity

In [ ]:
display(train.groupby(by='activity_name')[['heart_rate','chest_temperature','hand_temperature',
    'ankle_temperature']].mean())
discard = ['activity_id','activity','time_stamp','id']# Columns to exclude from descriptive statistics

Creating table with only relevant columns

In [ ]:
train_trimmed = train[[i for i in train.columns if i not in discard]]

Descriptive info of relevant feature

In [ ]:
display(train_trimmed.describe())

Correlation table of relevant features

In [ ]:
display(train_trimmed.corr()) 

Variance of each axis of acceleration grouped by activities

In [ ]:
coordinates = [i for i in train.columns if 'acceleration' in i]
display(train.groupby(by='activity_name')[coordinates].var())

## Hypothesis Testing  

Based on the exploratory data analysis carried out, the following hypotheses are tested on  <br>
the test set:<br>
- Heart rate of moderate activities are greater than heart rate of light activities.<br>
- Heart rate of intense activities are greater than heart rate of light activities.<br>
- Chest acceleration along z axis is greater than that along x axis during lying.<br>
- Chest acceleration along x axis is greater than that along z axis during running.

Based on the EDA  we performed, it does not seem that the data is normally distributed. It is <br>
for this reason that Wilcoxon rank sum test was used to test the above hypothesis instead of the usual t-test which assumes that the samples follow a normal distribution.<br>
We test the above hypothesis using the confidence level of 5%.

### Hypothesis 1<br>
$H_0$(Null) : The heart rate during  moderate activities are the same or lower than that of light activities. <br>
$H_1$(Alternate) : The heart rate during moderate activities are likely to be higher during lying compared to light activities.  

In [ ]:
test1 = test[test.activity_type=='moderate'].heart_rate.dropna()# Heart rate of moderate activities with nan values dropped
test2 = test[test.activity_type=='light'].heart_rate.dropna()# Heart rate of light activities with nan values dropped
print(ranksums(test1,test2,alternative='greater'))

### Hypothesis 2<br>
$H_0$(Null) : The heart rate during intense activities are the same or lower than that of light activities. <br>
$H_1$(Alternate) : The heart rate during intense activities are likely to be higher during than during lower activities.  

In [ ]:
test1 = test[test.activity_type=='intense'].heart_rate.dropna()# Heart rate of moderate activities with nan values dropped
test2 = test[test.activity_type=='light'].heart_rate.dropna()# Heart rate of light activities with nan values dropped
print(ranksums(test1,test2,alternative='greater'))

Since we get a p-value of 0 which is lower than 0.05 we reject the null hypothesis and accept<br>
the alternate hypothesis. 

### Hypothesis 3<br>
$H_0$(Null) : The z axis chest acceleration during lying is lower or same as the x axis acceleration. <br>
$H_1$(Alternate) :The z axis chest acceleration during lying is higher than the x axis acceleration. 

In [ ]:
test_l = test[test.activity_name=='lying']
feature1='chest_3D_acceleration_16_z'
feature2='chest_3D_acceleration_16_x'
test1 = test_l[feature1]
test2 = test_l[feature2]
print(ranksums(test1,test2,alternative='greater'))

Since we get a p-value of 0 which is lower than 0.05 we reject the null hypothesis and accept<br>
the alternate hypothesis. 

### Hypothesis 4<br>
$H_0$(Null) : The x axis chest acceleration during running is lower or same as the z axis acceleration. <br>
$H_1$(Alternate) :The x axis chest acceleration during lying is higher than the z axis acceleration. 

In [ ]:
test_l = test[test.activity_name=='running']
feature1='chest_3D_acceleration_16_x'
feature2='chest_3D_acceleration_16_z'
test1 = test_l[feature1]
test2 = test_l[feature2]
print(ranksums(test1,test2,alternative='greater'))

Since we get a p-value of 0 which is lower than 0.05 we reject the null hypothesis and accept<br>
the alternate hypothesis. 

## Model Prediction

In [3]:
clean_data = pd.read_pickle("clean_act_data.pkl")

In [24]:
train_subjects = [101,103,104,105]
def determine_axis(feat):
    if 'x' in feat:
        return 'x'
    elif 'y' in feat:
        return 'y'
    else:
        return 'z'
    
def create_sliding_window_corr(data,win_len):
   hand_coords = [f'hand_3D_acceleration_16_{i}' for i in ['x','y','z']] 
   chest_coords = [f'chest_3D_acceleration_16_{i}' for i in ['x','y','z']] 
   ankle_coords = [f'ankle_3D_acceleration_16_{i}' for i in ['x','y','z']] 
   def calc_corr(coords):
      for i in itertools.combinations(coords,2):
        ax1 = determine_axis(i[0])
        ax2 = determine_axis(i[1])
        coord_type = coords[0][0:5]
        if '_' in coord_type:
            coord_type = coord_type[:-1]
        data[f'{coord_type}_{ax1}_{ax2}_corr']=data[
        i[0]].rolling(win_len).corr(data[i[1]])
      return data
   data = calc_corr(hand_coords)
   data = calc_corr(chest_coords)
   data = calc_corr(ankle_coords) 
   return data
def create_sliding_window_feats(data,feats,win_len):
#    pca = PCA(n_components=1)
    
    
   for feat in feats:
       data[f'{feat}_roll_mean'] = data[feat].rolling(win_len).mean()
       data[f'{feat}_roll_median'] = data[feat].rolling(win_len).mean()
       data[f'{feat}_roll_var'] = data[feat].rolling(win_len).var()
#        data = data.dropna()
   return data
   
def train_test_split_acttype(features):
    global clean_data
    clean_data = clean_data.dropna()
    train = clean_data[clean_data.id.isin(train_subjects)]
    val = clean_data[clean_data.id.isin([102,106])]
    test = clean_data[clean_data.id.isin([107,108])]
    x_train = train[features]
    x_val = val[features]
    x_test = test[features]
    y_train = le.fit_transform(train.activity_type)
    y_val = le.fit_transform(val.activity_type)
    y_test = le.fit_transform(test.activity_type)
    return x_train,x_val,x_test,y_train,y_val,y_test
def train_test_split_actname(features):
    global clean_data
    clean_data = clean_data.dropna()
    train = clean_data[clean_data.id.isin(train_subjects)]
    val = clean_data[clean_data.id.isin([102,106])]
    test = clean_data[clean_data.id.isin([107,108])]
    x_train = train[features]
    x_val = val[features]
    x_test = test[features]
    y_train = le.fit_transform(train.activity_name)
    y_val = le.fit_transform(val.activity_name)
    y_test = le.fit_transform(test.activity_name)
    return x_train,x_val,x_test,y_train,y_val,y_test

In [7]:
acc_cols = [i for i in clean_data.columns if 'acceleration' in i] 

In [8]:

final=[]
for i in clean_data.id.unique():
   temp = clean_data[clean_data.id==i]
   temp = create_sliding_window_feats(temp,acc_cols,256)
   temp = create_sliding_window_corr(temp,256)

   final.append(temp)
clean_data = pd.concat(final) 
# print(clean_data[[i for i in clean_data.columns if 'roll' in i]].head())

In [16]:
x_train,x_val,x_test,y_train,y_val,y_test = train_test_split()

time_stamp  activity_id  heart_rate  hand_temperature  \
7518          83.56            1        93.0           30.8125   
7519          83.57            1        93.0           30.8125   
7520          83.58            1        93.0           30.8125   
7521          83.59            1        93.0           30.8125   
7522          83.60            1        93.0           30.8125   
...             ...          ...         ...               ...   
2872015       95.06           24       162.0           25.1250   
2872016       95.07           24       162.0           25.1250   
2872017       95.08           24       162.0           25.1250   
2872018       95.09           24       162.0           25.1250   
2872019       95.10           24       162.0           25.1250   

         hand_3D_acceleration_16_x  hand_3D_acceleration_16_y  \
7518                       6.01526                    5.10206   
7519                       6.09325                    5.06421   
7520                       5.97881                    5.10269   
7521                       5.93701                    4.91261   
7522                       6.01580                    4.98858   
...                            ...                        ...   
2872015                    4.99466                    6.01881   
2872016                    5.02764                    5.90369   
2872017                    5.06409                    5.71370   
2872018                    5.13914                    5.63724   
2872019                    5.00812                    5.40645   

         hand_3D_acceleration_16_z  hand_3D_gyroscope_x  hand_3D_gyroscope_y  \
7518                       5.80912            -0.034872             0.006905   
7519                       5.88739            -0.025281            -0.001689   
7520                       5.84705            -0.023323             0.016251   
7521                       5.77010            -0.011916             0.034801   
7522                       5.84799            -0.020812             0.016273   
...                            ...                  ...                  ...   
2872015                    5.59830            -0.289166            -0.110170   
2872016                    5.48372            -0.275411            -0.128358   
2872017                    5.48491            -0.289885            -0.126548   
2872018                    5.48629            -0.234417            -0.101485   
2872019                    5.02326            -0.260924            -0.093849   

         hand_3D_gyroscope_z  ...  ankle_3D_acceleration_16_z_roll_var  \
7518                0.017553  ...                                  NaN   
7519                0.003196  ...                                  NaN   
7520                0.015499  ...                                  NaN   
7521                0.013740  ...                                  NaN   
7522                0.014696  ...                                  NaN   
...                      ...  ...                                  ...   
2872015             0.238570  ...                            10.923580   
2872016             0.267409  ...                            10.840460   
2872017             0.281483  ...                            10.682665   
2872018             0.275497  ...                            10.657063   
2872019             0.266205  ...                            10.552733   

         hand_x_y_corr  hand_x_z_corr  hand_y_z_corr  chest_x_y_corr  \
7518               NaN            NaN            NaN             NaN   
7519               NaN            NaN            NaN             NaN   
7520               NaN            NaN            NaN             NaN   
7521               NaN            NaN            NaN             NaN   
7522               NaN            NaN            NaN             NaN   
...                ...            ...            ...             ...   
2872015      -0.427479       0.611086      -0.663470        0.500456   
2872016  

In [25]:
le = preprocessing.LabelEncoder()
# roll_coll=[i for i in clean_data.columns if '_roll_' in i]
discard = ['activity_id','activity','activity_name','time_stamp', \
           'id','activity_type']# Columns to exclude from descriptive statistics
features = [i for i in clean_data.columns if i not in discard]
x_train,x_val,x_test,y_train,y_val,y_test = train_test_split_actname(
    features)

In [51]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10, random_state=0)
rf.fit(x_train,y_train)
print(classification_report(y_val,rf.predict(x_val)))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91     56425
           1       0.50      0.86      0.63     30633
           2       0.99      0.98      0.98     45594
           3       0.83      0.53      0.65     26485
           4       0.83      0.96      0.89     66624
           5       0.99      0.94      0.97     46260
           6       0.98      0.86      0.92     13518
           7       0.99      0.92      0.96     32063
           8       0.94      0.78      0.85     45386
           9       0.78      0.81      0.79     49932
          10       0.74      0.86      0.80     41761
          11       0.84      0.72      0.78     58254

    accuracy                           0.85    512935
   macro avg       0.87      0.84      0.84    512935
weighted avg       0.87      0.85      0.85    512935



In [50]:
from sklearn.ensemble import RandomForestClassifier
minmax = MinMaxScaler()
x_train_trans=pca.transform(x_train)
x_train_trans_scaled = minmax.fit_transform(x_train_trans)
x_val_trans = pca.transform(x_val)
x_val_trans_scaled = minmax.transform(x_val_trans)
x_test_trans = pca.transform(x_test)
x_test_trans_scaled = minmax.transform(x_test_trans)
lr = LogisticRegression(C=10.0)
lr.fit(x_train_trans,y_train)
print(classification_report(y_val,lr.predict(x_val_trans)))

              precision    recall  f1-score   support

           0       0.87      0.67      0.76     56425
           1       0.29      0.28      0.29     30633
           2       0.92      0.74      0.82     45594
           3       0.61      0.40      0.48     26485
           4       0.81      0.95      0.87     66624
           5       0.93      0.96      0.94     46260
           6       0.84      0.77      0.80     13518
           7       0.94      0.93      0.93     32063
           8       0.84      0.89      0.86     45386
           9       0.70      0.64      0.67     49932
          10       0.63      0.75      0.69     41761
          11       0.62      0.77      0.69     58254

    accuracy                           0.75    512935
   macro avg       0.75      0.73      0.73    512935
weighted avg       0.76      0.75      0.75    512935



In [43]:
print(classification_report(y_val,lr.predict(x_val_trans)))

              precision    recall  f1-score   support

           0       0.87      0.67      0.76     56425
           1       0.30      0.28      0.29     30633
           2       0.92      0.74      0.82     45594
           3       0.61      0.40      0.48     26485
           4       0.81      0.95      0.87     66624
           5       0.93      0.96      0.94     46260
           6       0.83      0.77      0.80     13518
           7       0.94      0.93      0.93     32063
           8       0.84      0.89      0.86     45386
           9       0.70      0.64      0.67     49932
          10       0.63      0.75      0.69     41761
          11       0.62      0.77      0.69     58254

    accuracy                           0.76    512935
   macro avg       0.75      0.73      0.73    512935
weighted avg       0.76      0.76      0.75    512935



In [39]:
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.sum())

[0.61165123 0.07970362 0.06070157 0.0540126  0.04214078 0.03507637
 0.01710373 0.01634355 0.01401554 0.01313101 0.01158396]
0.9554639615348971


In [37]:
x_train.shape

(927138, 67)

In [26]:
s = pd.Series(y_train)
s.value_counts()

11    115254
4     109577
8     104597
5      94957
9      89088
10     87748
0      74069
2      70851
1      57254
3      57186
7      45912
6      20645
dtype: int64